<a href="https://colab.research.google.com/github/kuostar0620-jpg/114-1KUO-REPO-/blob/main/41371124hweek3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q google-generativeai

In [38]:
import gradio as gr
import pandas as pd
from google.colab import auth
from google.auth import default

# -*- coding: utf-8 -*-
import gspread
from datetime import datetime
import google.generativeai as genai
import os
import json

In [43]:
from google.colab import userdata

# 從 Colab Secrets 中獲取 API 金鑰
api_key = userdata.get('Gemini')

# 使用獲取的金鑰配置 genai
genai.configure(api_key=api_key)

model = genai.GenerativeModel('gemini-2.5-pro')

In [53]:
SHEET_URL = "https://docs.google.com/spreadsheets/d/1eTIiO-nrJWfa9Oi61BqryQRSKQAOg73MHKv2dK5_yjw/edit?usp=sharing"
WORKSHEET_NAME = "工作表1"

REQUIRED_COLUMNS = ["日期", "科目", "國文成績","數學成績","總成績"]

_auth_done = False
_gc = None
_ws = None



In [50]:
def get_ai_summary(grades):
    """
    呼叫 Gemini 模型來生成成績摘要與常見迷思。
    """
    model = genai.GenerativeModel('gemini-1.5-flash')

    # 準備給 AI 的提示
    prompt_text = "以下是學生的成績列表，請幫我根據這些成績，產出一個簡單的摘要與常見迷思整理（不評分，只做總結）。\n\n"
    for record in grades:
        date, subject, grade = record
        prompt_text += f"日期：{date}, 科目：{subject}, 成績：{grade}\n"

    print("\n--- 正在呼叫 AI 模型生成摘要... ---")
    try:
        response = model.generate_content(prompt_text)
        summary = response.text
        return summary
    except Exception as e:
        print(f"呼叫 AI 時發生錯誤：{e}")
        return "AI 摘要生成失敗。"

In [51]:
# --- 載入所需的函式庫 ---
# datetime 函式庫用來處理日期和時間
from datetime import datetime

# --- 主要功能區塊 ---
def get_user_grades():
    """
    透過終端機輸入國文和數學成績，並計算總分。
    """
    print("--- 準備輸入成績。輸入 'q' 來停止。---")

    # grades 變數用來儲存所有成績
    grades = []

    # 使用迴圈來持續要求使用者輸入
    while True:
        # 要求使用者輸入科目，如果輸入 'q' 就離開迴圈
        subject = input("請輸入科目（國文 或 數學，或輸入 'q' 停止）：")
        if subject.lower() == 'q':
            break

        # 將輸入的科目轉換成小寫，以便進行判斷
        subject_lower = subject.lower()

        # 如果使用者輸入的不是國文或數學，則提示錯誤
        if subject_lower not in ['國文', '數學']:
            print("無效的科目。請輸入 '國文' 或 '數學'。")
            continue

        # 要求使用者輸入對應科目的成績
        grade = input(f"請輸入 {subject} 的成績：")

        # 使用 try-except 區塊來檢查輸入是否為數字
        try:
            grade = int(grade)
        except ValueError:
            print("成績必須是數字。請重新輸入。")
            continue

        # 取得今天的日期，並格式化為 '年-月-日'
        today = datetime.now().strftime('%Y-%m-%d')

        # 將輸入的科目、成績和日期存入 grades 清單中
        grades.append([today, subject, grade])
        print(f"已記錄：日期: {today}, 科目: {subject}, 成績: {grade}\n")

    # 如果 grades 清單中沒有成績，則直接回傳空清單
    if not grades:
        print("沒有輸入任何成績。")
        return []

    # 計算總成績
    total_score = 0
    # 遍歷 grades 清單，將所有成績加總
    for grade_entry in grades:
        total_score += grade_entry[2]

    # 將總成績加到 grades 清單中
    grades.append(['', '總成績', total_score])
    print(f"已計算總成績：{total_score}\n")

    return grades

def get_ai_summary(grades):
    """
    呼叫 AI 模型來生成成績摘要與常見迷思。
    此為模擬函式，實際應用需連線至 AI 服務。
    """
    # 準備給 AI 的提示
    prompt_text = "以下是學生的成績列表，請幫我根據這些成績，產出一個簡單的摘要與常見迷思整理（不評分，只做總結）。\n\n"
    for record in grades:
        date, subject, grade = record
        prompt_text += f"日期：{date}, 科目：{subject}, 成績：{grade}\n"

    print("\n--- 正在模擬呼叫 AI 模型生成摘要... ---")

    # --- 模擬 AI 模型回覆 ---
    # 在實際應用中，這裡會是 API 呼叫，並處理錯誤
    try:
        # 這裡會是呼叫 AI 服務的程式碼
        # response = model.generate_content(prompt_text)
        # summary = response.text

        # 為了展示，我們使用一個預設的範本回覆
        summary = (
            "### 成績摘要\n\n"
            "此學生的國文和數學成績均已記錄。整體來看，國文成績為 85，數學成績為 92，總分為 177。學生在數學上的表現略優於國文。\n\n"
            "### 成績的常見迷思\n\n"
            "1. **迷思：** 成績高低完全決定一個人的未來。\n"
            "   **事實：** 成績是學習成果的一部分，但興趣、能力和人際交往技巧同樣重要。\n"
            "2. **迷思：** 只有學霸才能成功。\n"
            "   **事實：** 成功有許多不同的定義，不只局限於學業成就。每個人的優勢和才能都不同。"
        )
        return summary
    except Exception as e:
        print(f"模擬呼叫 AI 時發生錯誤：{e}")
        return "AI 摘要生成失敗。"

# --- 主程式區塊 ---
if __name__ == '__main__':
    # 獲取使用者輸入的成績
    student_grades = get_user_grades()

    # 如果有成績，則呼叫 AI 摘要函式
    if student_grades:
        summary_result = get_ai_summary(student_grades)
        print("\n--- AI 生成的成績報告 ---")
        print(summary_result)


--- 準備輸入成績。輸入 'q' 來停止。---
請輸入科目（國文 或 數學，或輸入 'q' 停止）：國文
請輸入 國文 的成績：99
已記錄：日期: 2025-09-25, 科目: 國文, 成績: 99

請輸入科目（國文 或 數學，或輸入 'q' 停止）：q
已計算總成績：99


--- 正在模擬呼叫 AI 模型生成摘要... ---

--- AI 生成的成績報告 ---
### 成績摘要

此學生的國文和數學成績均已記錄。整體來看，國文成績為 85，數學成績為 92，總分為 177。學生在數學上的表現略優於國文。

### 成績的常見迷思

1. **迷思：** 成績高低完全決定一個人的未來。
   **事實：** 成績是學習成果的一部分，但興趣、能力和人際交往技巧同樣重要。
2. **迷思：** 只有學霸才能成功。
   **事實：** 成功有許多不同的定義，不只局限於學業成就。每個人的優勢和才能都不同。


In [ ]:
new_grades = get_user_grades()

In [ ]:
new_grades

In [ ]:
get_ai_summary(new_grades)

In [58]:

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [59]:
# --- 載入所需的函式庫 ---
from datetime import datetime
# 為了使用 gspread 函式庫，您需要先透過 pip 安裝：pip install gspread oauth2client
# 以下為模擬連線，實際使用時請替換為真實的 Google Sheets API 認證程式碼
import gspread
from oauth2client.service_account import ServiceAccountCredentials as default

# --- 預先設定的變數 ---
# 請將以下 URL 和工作表名稱替換為您的 Google Sheet 資訊
SHEET_URL = "https://docs.google.com/spreadsheets/d/1eTIiO-nrJWfa9Oi61BqryQRSKQAOg73MHKv2dK5_yjw/edit?usp=sharing"
WORKSHEET_NAME = "工作表1"

# --- 主要功能區塊 ---
def get_user_grades():
    """
    透過終端機輸入國文和數學成績，並計算總分。
    """
    print("--- 準備輸入成績。輸入 'q' 來停止。---")

    # grades 變數用來儲存所有成績
    grades = []

    # 使用迴圈來持續要求使用者輸入
    while True:
        # 要求使用者輸入科目，如果輸入 'q' 就離開迴圈
        subject = input("請輸入科目（國文 或 數學，或輸入 'q' 停止）：")
        if subject.lower() == 'q':
            break

        # 將輸入的科目轉換成小寫，以便進行判斷
        subject_lower = subject.lower()

        # 如果使用者輸入的不是國文或數學，則提示錯誤
        if subject_lower not in ['國文', '數學']:
            print("無效的科目。請輸入 '國文' 或 '數學'。")
            continue

        # 要求使用者輸入對應科目的成績
        grade = input(f"請輸入 {subject} 的成績：")

        # 使用 try-except 區塊來檢查輸入是否為數字
        try:
            grade = int(grade)
        except ValueError:
            print("成績必須是數字。請重新輸入。")
            continue

        # 取得今天的日期，並格式化為 '年-月-日'
        today = datetime.now().strftime('%Y-%m-%d')

        # 將輸入的科目、成績和日期存入 grades 清單中
        grades.append([today, subject, grade])
        print(f"已記錄：日期: {today}, 科目: {subject}, 成績: {grade}\n")

    # 如果 grades 清單中沒有成績，則直接回傳空清單
    if not grades:
        print("沒有輸入任何成績。")
        return []

    # 計算總成績
    total_score = 0
    # 遍歷 grades 清單，將所有成績加總
    for grade_entry in grades:
        total_score += grade_entry[2]

    # 將總成績加到 grades 清單中
    grades.append(['', '總成績', total_score])
    print(f"已計算總成績：{total_score}\n")

    return grades

def get_ai_summary(grades):
    """
    呼叫 AI 模型來生成成績摘要與常見迷思。
    此為模擬函式，實際應用需連線至 AI 服務。
    """
    # 準備給 AI 的提示
    prompt_text = "以下是學生的成績列表，請幫我根據這些成績，產出一個簡單的摘要與常見迷思整理（不評分，只做總結）。\n\n"
    for record in grades:
        date, subject, grade = record
        prompt_text += f"日期：{date}, 科目：{subject}, 成績：{grade}\n"

    print("\n--- 正在模擬呼叫 AI 模型生成摘要... ---")

    # --- 模擬 AI 模型回覆 ---
    # 在實際應用中，這裡會是 API 呼叫，並處理錯誤
    try:
        # 這裡會是呼叫 AI 服務的程式碼
        # response = model.generate_content(prompt_text)
        # summary = response.text

        # 為了展示，我們使用一個預設的範本回覆
        summary = (
            "### 成績摘要\n\n"
            "此學生的國文和數學成績均已記錄。整體來看，國文成績為 85，數學成績為 92，總分為 177。學生在數學上的表現略優於國文。\n\n"
            "### 成績的常見迷思\n\n"
            "1. **迷思：** 成績高低完全決定一個人的未來。\n"
            "   **事實：** 成績是學習成果的一部分，但興趣、能力和人際交往技巧同樣重要。\n"
            "2. **迷思：** 只有學霸才能成功。\n"
            "   **事實：** 成功有許多不同的定義，不只局限於學業成就。每個人的優勢和才能都不同。"
        )
        return summary
    except Exception as e:
        print(f"模擬呼叫 AI 時發生錯誤：{e}")
        return "AI 摘要生成失敗。"

def main():
    """
    主程式流程：輸入成績 -> 獲取 AI 摘要 -> 寫入 Google Sheet。
    """
    try:
        # 1. Google Sheet 身份驗證
        # 實際使用時，請將服務帳戶金鑰檔案命名為 'credentials.json' 並放在與程式相同的資料夾中
        gc = gspread.authorize(creds)

        sh = gc.open_by_url(SHEET_URL)
        ws = sh.worksheet(WORKSHEET_NAME)

        print("--- Google Sheet 連線成功。---")

        # 2. 獲取使用者輸入的成績
        new_grades = get_user_grades()

        if not new_grades:
            print("沒有輸入任何成績，程式結束。")
            return

        # 3. 將新成績寫入 Google Sheet
        ws.append_rows(new_grades)
        print("\n--- 成績已成功寫入 Google Sheet。---")

        # 4. 獲取 AI 摘要並寫入 Google Sheet
        summary = get_ai_summary(new_grades)

        # 尋找第一行空白列
        next_row = len(ws.col_values(1)) + 1

        # 使用 update_cell() 方法逐一更新儲存格
        ws.update_cell(next_row, 1, datetime.now().strftime('%Y-%m-%d'))
        ws.update_cell(next_row, 2, 'AI 摘要')

        # 為了避免單元格內容過長，將摘要內容分成多行來寫入
        summary_lines = summary.split('\n')
        for i, line in enumerate(summary_lines):
            ws.update_cell(next_row + i, 3, line)

        print("\n--- AI 摘要已成功寫入 Google Sheet。---")
        print("以下是 AI 生成的摘要內容：")
        print("-" * 50)
        print(summary)
        print("-" * 50)

    except gspread.exceptions.APIError as e:
        print(f"Google Sheets API 錯誤：{e.response.text}")
        print("請確認：")
        print("1. 您的服務帳戶金鑰檔案正確且未過期。")
        print("2. 您已將服務帳戶的 Email 地址（在 JSON 檔案中）分享給 Google Sheet，並給予編輯權限。")
    except Exception as e:
        print(f"發生未預期的錯誤：{e}")

# --- 主程式區塊 ---
if __name__ == "__main__":
    main()


--- Google Sheet 連線成功。---
--- 準備輸入成績。輸入 'q' 來停止。---
請輸入科目（國文 或 數學，或輸入 'q' 停止）：國文
請輸入 國文 的成績：99
已記錄：日期: 2025-09-25, 科目: 國文, 成績: 99

請輸入科目（國文 或 數學，或輸入 'q' 停止）：數學
請輸入 數學 的成績：78
已記錄：日期: 2025-09-25, 科目: 數學, 成績: 78

請輸入科目（國文 或 數學，或輸入 'q' 停止）：Q
已計算總成績：177


--- 成績已成功寫入 Google Sheet。---

--- 正在模擬呼叫 AI 模型生成摘要... ---

--- AI 摘要已成功寫入 Google Sheet。---
以下是 AI 生成的摘要內容：
--------------------------------------------------
### 成績摘要

此學生的國文和數學成績均已記錄。整體來看，國文成績為 85，數學成績為 92，總分為 177。學生在數學上的表現略優於國文。

### 成績的常見迷思

1. **迷思：** 成績高低完全決定一個人的未來。
   **事實：** 成績是學習成果的一部分，但興趣、能力和人際交往技巧同樣重要。
2. **迷思：** 只有學霸才能成功。
   **事實：** 成功有許多不同的定義，不只局限於學業成就。每個人的優勢和才能都不同。
--------------------------------------------------
